<img src='https://drive.google.com/uc?export=view&id=1jyAgAg3L5dUSfO9zfp_bXVZrlV8HM-Z9'>




# Import library, Set Working Directory and Login Info

## Install OpenSilex in the current Environment
https://github.com/OpenSILEX/opensilexClientToolsPython

In [1]:
!pip install git+https://github.com/OpenSILEX/opensilexClientToolsPython.git@1.3.3

  Cloning https://github.com/OpenSILEX/opensilexClientToolsPython.git (to revision 1.3.3) to /tmp/pip-req-build-fchn_5wp
  Running command git clone --filter=blob:none --quiet https://github.com/OpenSILEX/opensilexClientToolsPython.git /tmp/pip-req-build-fchn_5wp
  Running command git checkout -q 063817ff3badb7b6b9e0c7e74eb174fb06857616
  Resolved https://github.com/OpenSILEX/opensilexClientToolsPython.git to commit 063817ff3badb7b6b9e0c7e74eb174fb06857616
  Preparing metadata (setup.py) ... done
  Created wheel for opensilexClientToolsPython: filename=opensilexClientToolsPython-1.3.3-py3-none-any.whl size=946999 sha256=4a079aea107509c9fcd9c2339420858c044ff7346d7583fb3462bba65aedb355
  Stored in directory: /tmp/pip-ephem-wheel-cache-0xubkg5a/wheels/80/3a/25/4ca94fcb760820426e33c4d519b186a7d51fd10c71dab27601
Successfully built opensilexClientToolsPython


## Import all requiered library

In [2]:
### Import library ##########################################################################################################
import os
import opensilexClientToolsPython as osC
import pandas as pd
import sys
import datetime
import time
from lxml import etree as ET
import json
from tqdm import tqdm
import yaml
from google.colab import drive

print("Library Imported")

Library Imported


## Setting Up the Working Environment
Mount Google Drive (Enabling the recovery of data generated in previous sessions.)

Clone PHIS-Related Data from GitHub

Set the Working Directory


In [4]:
### Clone all Data from GitHub ##############################################################################################
!git clone https://github.com/NaPPI-T/2025_NBPPN

### Set Database wd #########################################################################################################
wd = '/content/2025_NBPPN'

Cloning into '2025_NBPPN'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 105 (delta 27), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (105/105), 251.89 MiB | 28.38 MiB/s, done.
Resolving deltas: 100% (27/27), done.


# Connect to the PHIS Demo instance
https://phis.emphasis.fedcloud.eu/egi-demo/app/

login: trainees@opensilex.org

passowrd: trainees

In [5]:
### Get Login Info ##########################################################################################################
with open (os.path.join(wd, '00-yaml', 'Demo-login.yaml'), 'r') as stream:
    login=yaml.safe_load(stream)

### Connect to PHIS Database ################################################################################################
Py_Client = osC.ApiClient()
Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                  password=login["Password"],
                                  host=login["Host"])
#print(Py_Client.default_headers['Authorization'])
if Py_Client.default_headers:
    print("Connected")

Connected


# Get Experiment Info
<img src='https://drive.google.com/uc?export=view&id=1dy_OQTIckBbUxPA1RndMSGR2cM8HVHAa'>


In [7]:
prefix='SP'

### Set Experiment Name #####################################################################################################
NameExp = prefix + "_PotatoExp01"
print(f'Experiment Name is "{NameExp}"\n')


### Get Exp Info ############################################################################################################
with open (os.path.join(wd, '00-yaml', 'ExpInfo.yaml'), 'r') as stream:
    ExpInfo=yaml.safe_load(stream)

display(ExpInfo)

Experiment Name is "SP_PotatoExp01"



{'Objective': "Study the water stress response in 'Timo' potato cultivar",
 'Description': 'This experiment conducted in 2022 investigates the response of Timo cultivar to varying water levels to assess the plants physiological and adaptive responses to water stress. Plants were grown under two conditions: a control group maintained at 90% water holding capacity (WHC) and a stress group with WHC reduced to 70%.',
 'Start Date': datetime.date(2022, 6, 10),
 'End Date': datetime.date(2022, 7, 19),
 'organisations': ['National Plant Phenotyping Infrastructure'],
 'facilities': ['Modular'],
 'projects': ['DrTimo'],
 'scientific_supervisors': None,
 'technical_supervisors': None,
 'groups': ['NaPPI Team'],
 'Is_Public': False,
 'RDF Type': ['Plant']}

# Get or Create Experiment Name&URI

In [ ]:
### Get or Create Scientific Objects Name&URI ###############################################################################
Exp_Api = osC.ExperimentsApi(Py_Client)
Exp_Src = Exp_Api.search_experiments(name=NameExp)["result"]

NameExp_uri = {}
if Exp_Src:
    NameExp_uri.update({NameExp: Exp_Src[0].uri})
    print("Experiment URI:\n{}".format(NameExp_uri[NameExp]))
    del Exp_Api, Exp_Src

else:
    # List Exp Mandatory Info ###############################################################################################
    ObjectiveExp = ExpInfo['Objective']
    if ObjectiveExp != None:
      print("Objective: {}".format(ObjectiveExp))
    else:
      sys.exit("Objective Missing")

    StartExp = ExpInfo['Start Date']
    #print(ExpInfo['Start Date'])
    if StartExp != None:
      print("Start Date: {}".format(StartExp))
    else:
      sys.exit("Starting Date Missing")
    print('#'*50)

    # Get Description Name&URI ##############################################################################################
    DescriptionExp = ExpInfo['Description']
    print("Description: {}".format(str(DescriptionExp)))

    # Get End Date Name&URI #################################################################################################
    EndExp = ExpInfo['End Date']
    #print("End Date: {}".format(EndExp))
    if EndExp != None:
      print("Start Date: {}".format(EndExp))
    else:
      print("Ending Date Missing")

    # Get Is_Public Info ####################################################################################################
    Is_Public = ExpInfo['Is_Public']
    print("Is_Public: {}".format(str(Is_Public)))
    print('#'*50)

    # Get Organisation Name&URI #############################################################################################
    Org_Api = osC.OrganizationsApi(Py_Client)
    Organisation = ExpInfo['organisations']
    #print("Organization: {}".format(str(Organisation)))
    Organisation_uri = {}
    if Organisation == None:
        print("Organisation Missing")
        ls_Organisation=None
    else:
      Organisation = Organisation.split(", ")
      print("Organisation: {}".format(str(Organisation)))
      for organisation in Organisation:
        Org_Src = Org_Api.search_organizations(pattern=organisation)["result"]
        if Org_Src:
          Organisation_uri.update({organisation: Org_Src[0].uri})
          print("{} URI: {}".format(organisation, Org_Src[0].uri))
          ls_Organisation=list(Organisation_uri.values())
        else:
          print("{}: Unknown Organisation".format(organisation))
        del organisation, Org_Src
    del Org_Api, Organisation
    print('#'*50)

    # Get Groups Name&URI ###################################################################################################
    Sec_Api = osC.SecurityApi(Py_Client)
    Groups = ExpInfo['groups']
    #print("Groups: {}".format(str(Groups)))
    Groups_uri = {}
    if Groups == None:
        print("Group Missing")
        ls_Groups=None
    else:
      Groups = Groups.split(", ")
      for group in Groups:
        Sec_Src = Sec_Api.search_groups(name=group)["result"]
        if Sec_Src:
          Groups_uri.update({group: Sec_Src[0].uri})
          print("{} URI: {}".format(group, Sec_Src[0].uri))
          ls_Groups=list(Groups_uri.values())
        else:
          print("{}: Unknown Group".format(group))
          del group, Sec_Src
    del Sec_Api, Groups
    print('#'*50)

    # Get Scientific Supervisor Name&URI ####################################################################################
    Sec_Api = osC.SecurityApi(Py_Client)
    Scientific_Supervisors = ExpInfo['scientific_supervisors']
    #print("Scientific Supervisors: {}".format(str(Scientific_Supervisors)))
    Scientific_Supervisors_uri = {}
    if Scientific_Supervisors == None:
        print("Scientific Supervisors Missing")
        ls_Scientific_Supervisors=None
    else:
      Scientific_Supervisors = Scientific_Supervisors.split(", ")
      for scisup in Scientific_Supervisors:
        Sec_Src = Sec_Api.search_users(name=scisup)["result"]
        if Sec_Src:
          Scientific_Supervisors_uri.update({scisup: Sec_Src[0].uri})
          print("{} URI: {}".format(scisup, Sec_Src[0].uri))
          ls_Scientific_Supervisors=list(Scientific_Supervisors_uri.values())
        else:
          print("{}: Unknown Scientific Supervisors".format(scisup))
          del scisup, Sec_Src
    del Sec_Api, Scientific_Supervisors
    print('#'*50)

    # Get Technical Supervisor Name&URI #####################################################################################
    Sec_Api = osC.SecurityApi(Py_Client)
    Technical_Supervisors = ExpInfo['technical_supervisors']
    #print("Technical Supervisors: {}".format(str(Technical_Supervisors)))
    Technical_Supervisors_uri = {}
    if Technical_Supervisors == None:
        print("Technical Supervisors Missing")
        ls_Technical_Supervisors=None

    else:
      Technical_Supervisors = Technical_Supervisors.split(", ")
      for techsup in Technical_Supervisors:
        Sec_Src = Sec_Api.search_users(name=techsup)["result"]
        if Sec_Src:
          Technical_Supervisors_uri.update({techsup: Sec_Src[0].uri})
          print("{} URI: {}".format(techsup, Sec_Src[0].uri))
          ls_Technical_Supervisors=list(Technical_Supervisors_uri.values())

        else:
          print("{}: Unknown Technical Supervisors".format(techsup))
          del techsup, Sec_Src
    del Sec_Api, Technical_Supervisors
    print('#'*50)

    # Create Experiment #####################################################################################################
    body = osC.ExperimentCreationDTO(
        name=NameExp,
        start_date=StartExp,
        end_date=EndExp,
        description=DescriptionExp,
        objective=ObjectiveExp,
        organisations=ls_Organisation,
        scientific_supervisors=ls_Scientific_Supervisors,
        technical_supervisors=ls_Technical_Supervisors,
        groups=ls_Groups,
        is_public=Is_Public)
    Api_Resp = Exp_Api.create_experiment(body=body,)
    print("Experiment Creation: {}".format(str(Api_Resp["metadata"]["datafiles"])))

    # Get Experiment Name&URI ###############################################################################################
    Exp_Src = Exp_Api.search_experiments(name=NameExp)
    NameExp_uri.update({NameExp: Exp_Src["result"][0].uri})
    print( "{} URI: {}".format(NameExp,  NameExp_uri[NameExp]))
    del Exp_Api, body, Api_Resp, DescriptionExp, ObjectiveExp, StartExp, EndExp, Is_Public, Organisation_uri, Groups_uri, Scientific_Supervisors_uri, Technical_Supervisors_uri

# Reference Scientific Object
Your observed object (e.g., Plant, Leaf, Plot) is linked to several metadata elements that provide essential context and description:



*   Has Germplasm: Specifies the genetic material or cultivar used in the
experiment.
*   Has Factor Level: Describes the specific treatment or environmental condition applied (e.g., drought level, fertilizer type, temperature).
*   Has Facility: Indicates the locationl or controlled environment where the observation was conducted.
*   ...

These metadata help ensure that the scientific object is fully described.

<img src='https://drive.google.com/uc?export=view&id=1Kp79lO6dFdZBorh_Qy1tnjsUBug4g76c'>

## Get or Create Germplasms Name&URI

In [ ]:
### Get Germplasms Info ######################################################################################################
with open (os.path.join(wd, 'yaml', 'Germplasm.yaml'), 'r') as stream:
    Germplasms=yaml.safe_load(stream)

display(Germplasms)

In [ ]:
### Get Subtaxa Name&URI ####################################################################################################
Germ_Api = osC.GermplasmApi(Py_Client)

Species_uri={}

for germplasm, rdftype in Germplasms.items():
  Germ_Src = Germ_Api.search_germplasm(name=germplasm,
                                       rdf_type=rdftype)["result"]
  if Germ_Src:
    Species_uri.update({germplasm: Germ_Src[0].uri})
    print("{}, {} URI: {}".format(germplasm, rdftype, Species_uri[germplasm]))

  else:
    check_only = False
    body = osC.GermplasmCreationDTO(
        name=germplasm,
        rdf_type=rdftype)

    Germ_Api.create_germplasm(body=body, check_only=check_only)
    Germ_Src = Germ_Api.search_germplasm(name=germplasm,
                                         rdf_type=rdftype)["result"]
    Species_uri.update({germplasm: Germ_Src[0].uri})
    print("{}, {} Creation: {}".format(germplasm, rdftype, Germ_Src[0].uri))
del germplasm, Germ_Src

## Get or Create Factors&Levels Names&URI
<img src='https://drive.google.com/uc?export=view&id=1UPxaNYLNoU-27CvBAZmMOlqaJDHaEIAR'>

In [ ]:
### Get Exp Info ############################################################################################################
with open (os.path.join(wd, 'yaml', 'Factors.yaml'), 'r') as stream:
    Factors=yaml.safe_load(stream)

display(Factors)

In [ ]:
### Search Factors ##########################################################################################################
Fac_Api = osC.FactorsApi(Py_Client)
Factors_uri = {}
Factors_Levels_uri = {}
for factor in Factors:
    Fac_Src = Fac_Api.search_factors(name=factor,
                                     experiment=NameExp_uri[NameExp])["result"]

    # Get Existing Factors Name&URI #########################################################################################
    if Fac_Src:
        Factors_uri.update({factor: Fac_Src[0].uri})
        del Fac_Src

    # Create Factors ########################################################################################################
    else:
        # Creation of LevelsDTO #############################################################################################
        lvls = {}
        for i in Factors:
            lvl = []
            for j in Factors[i]['Levels']:
                lvl.append(osC.FactorLevelCreationDTO(name=j))
            lvls[i] = lvl

        # Creation of FactorDTO #############################################################################################
        bodies = []
        for i in lvls:
            body = osC.FactorCreationDTO(name=i,
                                         levels=lvls[i],
                                         experiment=NameExp_uri[NameExp],
                                         description=Factors[i]['Description'])
            bodies.append(body)

        # Creation of Factors ###############################################################################################
        for i in bodies:
            Api_Resp = Fac_Api.create_factor(body=i,)
            print("Factors Creation: {}".format(str(Api_Resp["metadata"]["datafiles"])))
        del lvls, lvl, bodies, body, Api_Resp, Fac_Src, i, j

        # Get New Factors Name&URI ##########################################################################################
        Fac_Src = Fac_Api.search_factors(name=factor,
                                         experiment=NameExp_uri[NameExp])["result"]
        Factors_uri.update({factor: Fac_Src[0].uri})

## Get Factors Levels Name&URI ##############################################################################################
for fac_uri in Factors_uri.values():
    Fac_Get = Fac_Api.get_factor_levels(uri=fac_uri)["result"]
    for lvl in Fac_Get:
        Factors_Levels_uri.update({lvl.name: lvl.uri})
del factor, fac_uri, Fac_Get, lvl

## Print Factors Name&URI ###################################################################################################
for factor in Factors_uri:
    print("{} URI: {}".format(factor, Factors_uri[factor]))
for lvl in Factors_Levels_uri:
    print("{} URI: {}".format(lvl, Factors_Levels_uri[lvl]))
del factor, lvl, Fac_Api, Factors

## Get Metadata & Numerical Data generated by the CNN Model

In [ ]:
### Get Metadata & Data ####################################################################################################
df_meta = pd.read_csv(os.path.join(wd, 'yaml', 'Metadata.csv'))
df_meta['ScObj Name'] = prefix + '_' + df_meta['image_id']

display(df_meta.head())

df_predic = pd.read_csv(os.path.join(wd, 'detection_summary.csv'))
df_predic['ScObj Name'] = prefix + '_' + df_predic['image_id']

display(df_predic.head())

df=pd.merge(df_meta, df_predic, on=['image_id', 'ScObj Name'])
display(df.head())

## Get or Create Scientific Objects Name&URI
<img src='https://drive.google.com/uc?export=view&id=1Nb9NzE8TfUAbIwHifj-fwtIWbmHVfwho'>

<img src='https://drive.google.com/uc?export=view&id=1uhCIhLu9mu96e8FXhJWzTdonbV5eUW9B'>


In [ ]:
df_meta.head()

In [ ]:
### Get or Create Scientific Objects Name&URI ###############################################################################
Relations_Gen = []

### ObjectRealtionDTO for Start Date ########################################################################################
StartExp = ExpInfo['Start Date']
if StartExp != None:
  relation_temp = osC.RDFObjectRelationDTO(
      _property="vocabulary:hasCreationDate",
      value=StartExp)
  Relations_Gen.append(relation_temp)
  del relation_temp, StartExp

else:
  print('Start Date Missing')

EndExp = ExpInfo['End Date']
if EndExp != None:
  EndExp = datetime.strptime(EndExp, "%d/%m/%Y")
  relation_temp = osC.RDFObjectRelationDTO(
      _property="vocabulary:hasDestructionDate",
      value=EndExp)
  Relations_Gen.append(relation_temp)
  del relation_temp, EndExp
else:
  print('End Date Missing')

### Scientific Object RDF Type ##############################################################################################
BioMat_Type = ExpInfo['RDF Type']
if not BioMat_Type:
    sys.exit("Scientific Object RDF Type Missing")
else:
    for biomat in BioMat_Type:
        Onto_Api = osC.OntologyApi(Py_Client)
        Onto_Src = Onto_Api.search_sub_classes_of(
            name=biomat, parent_type="vocabulary:ScientificObject")["result"]
        if Onto_Src:
            rdf_type = Onto_Src[0].children[0].uri
        else:
            sys.exit("Scientific Object RDF Type Unknown")
    del biomat, BioMat_Type, Onto_Api, Onto_Src

### Get Scientific Objects Name&URI #########################################################################################
ScObj_Api = osC.ScientificObjectsApi(Py_Client)
ScObj_uri = {}
for index, row in tqdm(df_meta.iterrows(), desc="ScObj processing:"):
    ScObj_Src = ScObj_Api.search_scientific_objects(name=row["ScObj Name"])["result"]
    if ScObj_Src:
        ScObj_uri.update({row["ScObj Name"]: ScObj_Src[0].uri})
    else:
        Relations_ScObj=[]

        # ObjectRealtionDTO for Germplasm ###################################################################################
        if Species_uri:
            relation_temp = osC.RDFObjectRelationDTO(
                _property="vocabulary:hasGermplasm",
                value=Species_uri.get(row["Species"]))
            Relations_ScObj.append(relation_temp)
            del relation_temp

        # ObjectRealtionDTO for Factors #####################################################################################
        if Factors_Levels_uri:
            Factors_temp = {"Factors": row["Plant Info"].split("_")}
            for factor in Factors_temp["Factors"]:
                relation_temp = osC.RDFObjectRelationDTO(
                    _property="vocabulary:hasFactorLevel",
                    value=Factors_Levels_uri.get(factor))
                Relations_ScObj.append(relation_temp)
                del relation_temp
        Relations = Relations_Gen + Relations_ScObj

        # Creation of Scientific Object #####################################################################################
        body = osC.ScientificObjectCreationDTO(name=row["ScObj Name"],
                                               rdf_type=rdf_type,
                                               relations=Relations,
                                               experiment=NameExp_uri[NameExp])
        ScObj_Api.create_scientific_object(body, )
        del Relations, Relations_ScObj

        # Get New Scientific Object Name&URI ################################################################################
        ScObj_Src = ScObj_Api.search_scientific_objects(name=row["ScObj Name"])["result"]
        ScObj_uri.update({row["ScObj Name"]: ScObj_Src[0].uri})
print("Done")
del index, row, ScObj_Src, ScObj_Api, Relations_Gen, rdf_type

display(ScObj_uri)

# Reference Provenance
<img src='https://drive.google.com/uc?export=view&id=1u57goNfGK7xlHeD4YcEs6TBF6jIM71h0'>

## Get or Create Original Images Provenance Name&URI

In [ ]:
prov_dict={}

### Get or Create Original Images Provenance ################################################################################
Dat_Api = osC.DataApi(Py_Client)

prov = "Canon-OriginalImages"

Prov_Src = Dat_Api.search_provenance(name=prov,)["result"]
if Prov_Src:
    prov_dict.update({prov: Prov_Src[0].uri})
    print("{} URI: {}".format(prov,Prov_Src[0].uri))
else:
    description = "Images acquired by a Canon DSLR Camera"

    prov_activity = [osC.ActivityCreationDTO(rdf_type="vocabulary:ImageAcquisition")]

    prov_agent = [

        osC.AgentModel(uri="http://phis.egi-demo.eu/id/device/field_camera",
                                 rdf_type="vocabulary:SensingDevice",
                                 settings={})]

    body = osC.ProvenanceCreationDTO(name=prov,
                                     description=description,
                                     prov_agent=prov_agent,
                                     prov_activity=prov_activity)

    Api_Resp = Dat_Api.create_provenance(body=body, )
    print("Provenance Created: {}".format(str(Api_Resp["metadata"]["datafiles"])))
    Prov_Src = Dat_Api.search_provenance(name=prov,)["result"]
    prov_dict.update({prov: Prov_Src[0].uri})
    print("{} URI Created: {}".format(prov,  Prov_Src[0].uri))
    del description, prov_activity, prov_agent, body, Api_Resp
del prov, Dat_Api, Prov_Src

## Get or Create BBoxed Images Provenance Name&URI

In [ ]:
### Get or Create Leaf_CNN BBox Images Provenance ###########################################################################
Dat_Api = osC.DataApi(Py_Client)

prov = 'Leaf_CNN-BBoxImage'

Prov_Src = Dat_Api.search_provenance(name=prov,)["result"]
if Prov_Src:
    prov_dict.update({prov: Prov_Src[0].uri})
    print("{} URI: {}".format(prov, Prov_Src[0].uri))
else:
    description = "Images annotated with Bounding Box predicted with a CNN"

    prov_activity = [osC.ActivityCreationDTO(rdf_type="vocabulary:ImageAnalysis")]

    prov_agent = [
        osC.AgentModel(uri="http://phis.egi-demo.eu/id/device/leaf_cnn-v1",
                       rdf_type="vocabulary:Software"),

        osC.AgentModel(uri="http://phis.egi-demo.eu/id/device/field_camera",
                                 rdf_type="vocabulary:SensingDevice",
                                 settings={})]

    body = osC.ProvenanceCreationDTO(name=prov,
                                     description=description,
                                     prov_agent=prov_agent,
                                     prov_activity=prov_activity)

    Api_Resp = Dat_Api.create_provenance(body=body, )
    print("Provenance Created: {}".format(str(Api_Resp["metadata"]["datafiles"])))
    Prov_Src = Dat_Api.search_provenance(name=prov,)["result"]
    prov_dict.update({prov: Prov_Src[0].uri})
    print("{} URI Created: {}".format(prov, Prov_Src[0].uri))
    del description, prov_activity, prov_agent, body, Api_Resp
del prov, Dat_Api, Prov_Src

## Get or Create Data Provenance Name&URI

In [ ]:
### Get or Create Leaf_CNN Data Provenance ##################################################################################
Dat_Api = osC.DataApi(Py_Client)

prov = 'Leaf_CNN-PredictedData'

Prov_Src = Dat_Api.search_provenance(name=prov,)["result"]
if Prov_Src:
    prov_dict.update({prov: Prov_Src[0].uri})
    print("{} URI: {}".format(prov, Prov_Src[0].uri))
else:
    description = "Data generated by CNN including Leaf Count, Coverage Height and Coverage Width"
    prov_activity = [osC.ActivityCreationDTO(rdf_type="vocabulary:ImageAnalysis")]
    prov_agent = [

        osC.AgentModel(uri="http://phis.egi-demo.eu/id/device/leaf_cnn-v1",
                       rdf_type="vocabulary:Software")
        ]

    body = osC.ProvenanceCreationDTO(name=prov,
                                     description=description,
                                     prov_agent=prov_agent,
                                     prov_activity=prov_activity)

    Api_Resp = Dat_Api.create_provenance(body=body, )
    print("Provenance Created: {}".format(str(Api_Resp["metadata"]["datafiles"])))
    Prov_Src = Dat_Api.search_provenance(name=prov,)["result"]
    prov_dict.update({prov: Prov_Src[0].uri})
    print("{} URI Created: {}".format(prov, Prov_Src[0].uri))
    del description, prov_activity, prov_agent, body, Api_Resp
del prov, Dat_Api, Prov_Src

# Import datafiles (Images)
This section handles the import of all image data files.

## List all images

In [ ]:
### List Original Images ####################################################################################################

wd_img = os.path.join(wd, 'Leaf','test', 'leaf')

ls_img = []
for (root, dirs, files) in os.walk(wd_img):
    for filename in files:
        if filename.endswith(".jpg"):
            ls_img.append(os.path.join(root, filename))

print(f'Numbers of Original Img: {len(ls_img)}')

### List BBoxed Images ######################################################################################################
wd_bbox = os.path.join(wd, 'runs','detect', 'predict')

ls_bbox = []
for (root, dirs, files) in os.walk(wd_bbox):
    for filename in files:
        if filename.endswith(".jpg"):
            ls_bbox.append(os.path.join(root, filename))

print(f'Numbers of BBoxed Img: {len(ls_bbox)}')

## Get Original Images Metadata
<img src='https://drive.google.com/uc?export=view&id=1mWgGuYDArYOAvDCwNQtuPznnA7PzV17R'>

In [ ]:
prov='Canon-OriginalImages'
### Initialize an empty list to store all the dictionaries ##################################################################
Img_Name_All = []
### Loop through each file in ls_fec ########################################################################################
for img in ls_img:

    # Extract the base filename and remove the ".jpg" extension #############################################################
    target = os.path.basename(img).replace(".jpg", "")

    # Create the dictionary with common keys ################################################################################
    Img_Dict = {
        "Path": img,
        "Target": prefix + '_' + target,
        'Prov': prov_dict[prov],
    }

    # Append the dictionary to the list #####################################################################################
    Img_Name_All.append(Img_Dict)

### Display the first dictionary in the list ################################################################################
display(Img_Name_All[0])
print(len(Img_Name_All))

## Check if any Original Images already exist

In [ ]:
prov='Canon-OriginalImages'

### Filter out existing Original images #####################################################################################
Dat_Api = osC.DataApi(Py_Client)
Dat_Src = Dat_Api.get_data_file_descriptions_by_search(provenances=[prov_dict[prov]], experiments=[NameExp_uri[NameExp]], page_size=100000)["result"]

Img_Name_Com=[]
for i in Img_Name_All:
    for elts in Dat_Src:
        if ScObj_uri[i["Target"]] == elts.target:
            Img_Name_Com.append(i)
        del elts
del i
print("{} images over {} already exit on the database".format(len(Img_Name_Com), len(Img_Name_All)))

### Exclude existing one from the Dict ######################################################################################
Img_Name=[]
for i in Img_Name_All:
        if i not in Img_Name_Com:
            Img_Name.append(i)
del i, Img_Name_All, Img_Name_Com, Dat_Src

## Import Original Images
<img src='https://drive.google.com/uc?export=view&id=1Qy0wxFlH60aC6Tlwz-mgELosErg3XYTj'>

In [ ]:
### Import Original Images ##################################################################################################
Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                  password=login["Password"],
                                  host=login["Host"])

timelimit = datetime.datetime.now() + datetime.timedelta(minutes=30)

Dat_Api = osC.DataApi(Py_Client)
for img in tqdm(Img_Name):
    description={"rdf_type": "vocabulary:RGBImage", #<- Here a DTO object was orginially proposed
                 "date": str(ExpInfo['Start Date']),
                 "target": ScObj_uri[img["Target"]],
                 "provenance": {"uri": img["Prov"],
                               "experiments": [NameExp_uri[NameExp]]
                               }
                }
    Dat_Api.post_data_file(description=json.dumps(description), file=img["Path"], ) #<- DTO object was replaced by json.dumps()
    #display(description)
    #print('#'*100)
    del img, description

if datetime.datetime.now() > timelimit:
    Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                      password=login["Password"],
                                      host=login["Host"])
    Dat_Api = osC.DataApi(Py_Client)
    timelimit = datetime.datetime.now()+datetime.timedelta(minutes=30)
    #print(Py_Client.default_headers['Authorization'])

print ('Done')

## Get Original datafile URI

In [ ]:
prov='Canon-OriginalImages'

### Recover Original URI ####################################################################################################
Dat_Api = osC.DataApi(Py_Client)
Dat_Src = Dat_Api.get_data_file_descriptions_by_search(provenances=[prov_dict[prov]], experiments=[NameExp_uri[NameExp]], page_size=100000)["result"]

Img_uri=[]
for elts in Dat_Src:
    for k, v in ScObj_uri.items():
        if v == elts.target:
            target=k

    Img_uri.append({'Type': 'Original',
                     "Target URI": [value for key, value in ScObj_uri.items() if key == target][0],
                     "Target Name": target,
                     "uri": elts.uri})
del elts, k, v, Dat_Src, Dat_Api

display(Img_uri[0])

## Get Bounding Boxed Images Metadata
<img src='https://drive.google.com/uc?export=view&id=14WEchn4gDIp4ZY-iWlXBBb1oU9se8jEm'>




In [ ]:
prov='Leaf_CNN-BBoxImage'

### Initialize an empty list to store all the dictionaries ##################################################################
BBox_Name_All = []

### Loop through each file in ls_fec ########################################################################################
for bbox in ls_bbox:
    # Extract the base filename and remove the ".jpg" extension #############################################################
    target = os.path.basename(bbox).replace(".jpg", "")

    # Create the dictionary with common keys ################################################################################
    BBox_Dict = {
        "Path": bbox,
        "Target": prefix + '_' + target,
        'Prov': prov_dict[prov],
    }

    # Append the dictionary to the list #####################################################################################
    BBox_Name_All.append(BBox_Dict)

for elts in BBox_Name_All:
    for item in Img_uri:
        if item["Target Name"]==elts["Target"]:
            elts.update({"Prov_Used": item["uri"]})

### Display the first dictionary in the list ################################################################################
display(BBox_Name_All[0])
print(len(BBox_Name_All))

## Check if any Bounding Boxed Images already exist

In [ ]:
prov='Leaf_CNN-BBoxImage'
### Filter out existing BBoxed Images #######################################################################################
Dat_Api = osC.DataApi(Py_Client)
Dat_Src = Dat_Api.get_data_file_descriptions_by_search(provenances=[prov_dict[prov]], experiments=[NameExp_uri[NameExp]], page_size=100000)["result"]

BBox_Name_Com=[]
for i in BBox_Name_All:
    for elts in Dat_Src:
        if ScObj_uri[i["Target"]] == elts.target:
            BBox_Name_Com.append(i)
        del elts
del i
print("{} images over {} already exit on the database".format(len(BBox_Name_Com), len(BBox_Name_All)))

### Exclude existing one from the Dict ######################################################################################
BBox_Name=[]
for i in BBox_Name_All:
        if i not in BBox_Name_Com:
            BBox_Name.append(i)
del i, BBox_Name_All, BBox_Name_Com, Dat_Src

## Import Bounding Boxed Images
<img src='https://drive.google.com/uc?export=view&id=1PNV2HRbgdCR1gPXWo6FHSbNOLWBu119q'>

In [ ]:
### Import BBoxed Images ####################################################################################################
Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                  password=login["Password"],
                                  host=login["Host"])

timelimit = datetime.datetime.now() + datetime.timedelta(minutes=30)

Dat_Api = osC.DataApi(Py_Client)
for bbox in tqdm(BBox_Name):
    description={"rdf_type": "vocabulary:RGBImage", #<- Here a DTO object was orginially proposed
                 "date": str(ExpInfo['Start Date']),
                 "target": ScObj_uri[bbox["Target"]],
                 "provenance": {"uri": bbox["Prov"],
                                "prov_used": [{"uri": bbox["Prov_Used"],
                                               "rdf_type":"vocabulary:RGBImage"}],
                               "experiments": [NameExp_uri[NameExp]]
                               }
                }
    Dat_Api.post_data_file(description=json.dumps(description), file=bbox["Path"], ) #<- DTO object was replaced by json.dumps()
    #display(description)
    #print('#'*100)
    del bbox, description

if datetime.datetime.now() > timelimit:
    Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                      password=login["Password"],
                                      host=login["Host"])
    Dat_Api = osC.DataApi(Py_Client)
    timelimit = datetime.datetime.now()+datetime.timedelta(minutes=30)
    #print(Py_Client.default_headers['Authorization'])

print ('Done')

## Get Bounding Box datafile URI

In [ ]:
prov='Leaf_CNN-BBoxImage'

### Recover BBoxed URI ######################################################################################################
Dat_Api = osC.DataApi(Py_Client)
Dat_Src = Dat_Api.get_data_file_descriptions_by_search(provenances=[prov_dict[prov]], experiments=[NameExp_uri[NameExp]], page_size=100000)["result"]

BBox_uri=[]
for elts in Dat_Src:
    for k, v in ScObj_uri.items():
        if v == elts.target:
            target=k

    BBox_uri.append({'Type': 'BBoxed',
                     "Target URI": [value for key, value in ScObj_uri.items() if key == target][0],
                     "Target Name": target,
                     "uri": elts.uri})
del elts, k, v, Dat_Src, Dat_Api

display(BBox_uri[0])

# Import Numerical Data

<img src='https://drive.google.com/uc?export=view&id=1u57goNfGK7xlHeD4YcEs6TBF6jIM71h0'>

## List Variable <-> Column Name Association  
This section establishes the mapping between observed variables and their corresponding column names in the dataset.

In [ ]:
### Get Exp Info ############################################################################################################
with open (os.path.join(wd, 'yaml', 'Variables.yaml'), 'r') as stream:
    Variables=yaml.safe_load(stream)

display(Variables)

## PHIS data import
This section handles the import of numerical data from the dataset.

In [ ]:
prov = 'Leaf_CNN-PredictedData'

### Import Numerical Data ###################################################################################################
Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                  password=login["Password"],
                                  host=login["Host"])
timelimit = datetime.datetime.now()+datetime.timedelta(minutes=30)

Dat_Api = osC.DataApi(Py_Client)
Var_Api = osC.VariablesApi(Py_Client)

Prov_Was_Associated_With=osC.ProvEntityModel(uri="phis-egi-demo:id/device/leaf_cnn-v1",
                                             rdf_type="vocabulary:Software")

logfile={}
for key, value in tqdm(Variables.items()):
    logfile[value] = []
    Var_Src = Var_Api.search_variables(name=value)["result"]
    pas=1000
    count=0
    for slc in range(0, len(df), pas):
        df_Slice = df.iloc[slc : slc + pas]
        bodies=[]
        count=count+1
        for index, row in df_Slice.iterrows():
            Dat_Src=Dat_Api.search_data_list(targets = [ScObj_uri[row["ScObj Name"]]],
                                             variables = [Var_Src[0].uri],
                                             experiments=[NameExp_uri[NameExp]], page_size=20)['result']
            if Dat_Src:
                logfile[value].append({'ScObj Name': {row["ScObj Name"]}})

            else:
                Prov_Used=None
                for item in BBox_uri:
                    if item["Target Name"]==row["ScObj Name"]:
                        Prov_Used=osC.ProvEntityModel(uri=item["uri"], rdf_type="vocabulary:RGBImage")
                body = osC.DataCreationDTO(_date = str(ExpInfo['Start Date']),
                                           target = ScObj_uri[row["ScObj Name"]],
                                           variable = Var_Src[0].uri,
                                           value = row[key],
                                           provenance = osC.DataProvenanceModel(
                                               uri = prov_dict[prov],
                                               prov_used = [Prov_Used],
                                               prov_was_associated_with = [Prov_Was_Associated_With],
                                               experiments = [NameExp_uri[NameExp]]))
                bodies.append(body)
                if datetime.datetime.now() > timelimit:
                    Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                                      password=login["Password"],
                                                      host=login["Host"])
                    Dat_Api = osC.DataApi(Py_Client)
                    timelimit = datetime.datetime.now()+datetime.timedelta(minutes=30)
        if bodies:
            #print(bodies)
            Dat_Api.add_list_data(body=bodies,)
        else:
            print(f'all data of {value} already uploaded')
print('Import Over')

<img src='https://drive.google.com/uc?export=view&id=1S-hsL8UKdHaGnuIBDhLQAzJmJ21pUug1'>


https://www.helsinki.fi/en/infrastructures/national-plant-phenotyping/hybrid-workshop-data-standards-plant-phenotyping